In [37]:
import os
import numpy as np
from keras.preprocessing import image
import cv2
from PIL import Image
from os import path
import shutil, copy
from image_cropper import *
from copy import deepcopy
from collections import defaultdict

In [46]:
def return_list(data_path, data_type):
    file_list = [file for file in os.listdir(data_path) if file.lower().endswith(data_type)]
    # print(str(len(file_list)))
    return file_list

def mk_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    return dir_path

source_data_type = '.png'
target_data_type = '.jpg'
TARGET_SIZE = 384

In [47]:
pad_classes = ['MEL', 'NEV', 'BCC', 'ACK']
# Malignant Melanoma (MEL)
# Melanocytic Nevus of Skin (NEV)
# Basal Cell Carcinoma of skin (BCC)
# Actinic Keratosis (ACK)
# Squamous Cell Carcinoma (SCC)
# Seborrheic Keratosis (SEK)

HamLabels = defaultdict(lambda: "NONE")
HamLabels["MEL"] = "MEL"
HamLabels["NEV"] = "NV"
HamLabels["BCC"] = "BCC"
HamLabels["ACK"] = "AKIEC"

In [48]:
metadata = pd.read_csv("../../datasets/PAD-UFES-20/metadata.csv")
target_metadata = deepcopy(metadata)
target_metadata['HamMappedLabel'] = HamLabels["DEFAULT"]

In [49]:
data_img_path_images = '../../datasets/PAD-UFES-20/images/'

data_save_path = '../../datasets/processed/PAD-UFES-20/roi_square_cropped/'
data_save_path_images = mk_dir(data_save_path + 'images/')

In [50]:
target_metadata.columns

Index(['patient_id', 'lesion_id', 'smoke', 'drink', 'background_father',
       'background_mother', 'age', 'pesticide', 'gender',
       'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'fitspatrick', 'region', 'diameter_1',
       'diameter_2', 'diagnostic', 'itch', 'grew', 'hurt', 'changed', 'bleed',
       'elevation', 'img_id', 'biopsed', 'HamMappedLabel'],
      dtype='object')

In [51]:
i=0
for image_name in list(target_metadata.img_id):
    i+=1
    row = target_metadata.loc[target_metadata.img_id == image_name]

    if HamLabels[row.diagnostic.values[0]] == "NONE":
        continue

    target_metadata.loc[target_metadata.img_id == image_name, "HamMappedLabel"] = HamLabels[row.diagnostic.values[0]]
    if i % 100 == 0:
        print('Processing Img {idx}: {image_name}'.format(idx=i, image_name=image_name))
    
    org_img = np.asarray(image.load_img(data_img_path_images + image_name))
    remove=min(org_img.shape[0], org_img.shape[1])//2
    x,y= org_img.shape[0]//2, org_img.shape[1]//2
    # centred square crop
    org_img = org_img[x-remove:x+remove, y-remove:y+remove]
    # resize
    org_img = cv2.resize(org_img, (TARGET_SIZE, TARGET_SIZE), interpolation = cv2.INTER_AREA)
   
    cropImg = Image.fromarray(org_img)
    cropImg.save(path.join(data_save_path_images, image_name[:-4] + target_data_type))

Processing Img 100: PAT_1710_3160_488.png
Processing Img 200: PAT_1638_2878_536.png
Processing Img 300: PAT_380_773_542.png
Processing Img 400: PAT_106_158_249.png
Processing Img 500: PAT_378_770_302.png
Processing Img 600: PAT_1704_3142_702.png
Processing Img 700: PAT_514_969_187.png
Processing Img 900: PAT_2103_4581_72.png
Processing Img 1000: PAT_2122_4642_204.png
Processing Img 1100: PAT_890_1694_29.png
Processing Img 1300: PAT_86_1082_220.png
Processing Img 1400: PAT_2026_4230_141.png
Processing Img 1500: PAT_138_206_507.png
Processing Img 1600: PAT_459_892_804.png
Processing Img 1700: PAT_154_236_800.png
Processing Img 1800: PAT_1443_1536_166.png
Processing Img 1900: PAT_1428_1489_454.png
Processing Img 2000: PAT_986_1855_702.png
Processing Img 2100: PAT_935_1781_920.png
Processing Img 2200: PAT_337_1697_721.png


In [53]:
target_metadata = target_metadata[target_metadata.HamMappedLabel != "NONE"]
target_metadata.to_csv(f"{data_save_path}target_metadata.csv", index=False)